In [2]:
## 라이브러리 추가하기
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [3]:
from torchvision import transforms, datasets

In [4]:
lr = 1e-3
batch_size = 64
num_epoch = 10

In [5]:
ckpt_dir = './checkpoint'
log_dir = './log'

In [6]:
# Device Setting 
device = torch.device('gpu' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cpu')

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()
        
        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)
        
        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        
        x = x.view(-1, 320)
        
        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)
        
        x = self.fc2(x)
        
        return x

In [9]:
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

In [10]:
def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

In [49]:
# Minist Dataset Load
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
dataset = datasets.MNIST(download=True, root='./',  train=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [50]:
num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

In [11]:
net = Net().to(device)
params = net.parameters()

In [12]:
fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

In [13]:
optim = torch.optim.Adam(params, lr=lr)
writer = SummaryWriter(log_dir=log_dir)

In [55]:
for epoch in range(1, num_epoch+1):
    net.train()
    
    loss_arr = []
    acc_arr = []
    
    for batch, (input_d, label) in enumerate(loader, 1):
        input_d = input_d.to(device)
        label = label.to(device)
        
        output = net(input_d)
        pred = fn_pred(output)
        
        optim.zero_grad()
        
        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)
        
        loss.backward()
        
        optim.step()
        
        loss_arr.append(loss.item())
        acc_arr.append(acc.item())
        
        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' % \
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))
    
    writer.add_scalar('loss', np.mean(loss_arr), epoch)
    writer.add_scalar('acc', np.mean(acc_arr), epoch)
    
    save(ckpt_dir = ckpt_dir, net=net, optim=optim, epoch=epoch)

writer.close()

C:\Users\khhh9\Anaconda3\envs\pytorch-tutorial\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


TRAIN: EPOCH 0001/0010 | BATCH 0001/0938 | LOSS: 2.3525 | ACC 0.0781
TRAIN: EPOCH 0001/0010 | BATCH 0002/0938 | LOSS: 2.3289 | ACC 0.0625
TRAIN: EPOCH 0001/0010 | BATCH 0003/0938 | LOSS: 2.3310 | ACC 0.0677
TRAIN: EPOCH 0001/0010 | BATCH 0004/0938 | LOSS: 2.3227 | ACC 0.0703
TRAIN: EPOCH 0001/0010 | BATCH 0005/0938 | LOSS: 2.3152 | ACC 0.0938
TRAIN: EPOCH 0001/0010 | BATCH 0006/0938 | LOSS: 2.3142 | ACC 0.0938
TRAIN: EPOCH 0001/0010 | BATCH 0007/0938 | LOSS: 2.3095 | ACC 0.0938
TRAIN: EPOCH 0001/0010 | BATCH 0008/0938 | LOSS: 2.3060 | ACC 0.0996
TRAIN: EPOCH 0001/0010 | BATCH 0009/0938 | LOSS: 2.3030 | ACC 0.1007
TRAIN: EPOCH 0001/0010 | BATCH 0010/0938 | LOSS: 2.3019 | ACC 0.1000
TRAIN: EPOCH 0001/0010 | BATCH 0011/0938 | LOSS: 2.2996 | ACC 0.0994
TRAIN: EPOCH 0001/0010 | BATCH 0012/0938 | LOSS: 2.2985 | ACC 0.0990
TRAIN: EPOCH 0001/0010 | BATCH 0013/0938 | LOSS: 2.2950 | ACC 0.1034
TRAIN: EPOCH 0001/0010 | BATCH 0014/0938 | LOSS: 2.2933 | ACC 0.1038
TRAIN: EPOCH 0001/0010 | BATCH 001

TRAIN: EPOCH 0001/0010 | BATCH 0120/0938 | LOSS: 1.5458 | ACC 0.4672
TRAIN: EPOCH 0001/0010 | BATCH 0121/0938 | LOSS: 1.5397 | ACC 0.4700
TRAIN: EPOCH 0001/0010 | BATCH 0122/0938 | LOSS: 1.5354 | ACC 0.4717
TRAIN: EPOCH 0001/0010 | BATCH 0123/0938 | LOSS: 1.5298 | ACC 0.4737
TRAIN: EPOCH 0001/0010 | BATCH 0124/0938 | LOSS: 1.5257 | ACC 0.4752
TRAIN: EPOCH 0001/0010 | BATCH 0125/0938 | LOSS: 1.5199 | ACC 0.4771
TRAIN: EPOCH 0001/0010 | BATCH 0126/0938 | LOSS: 1.5155 | ACC 0.4794
TRAIN: EPOCH 0001/0010 | BATCH 0127/0938 | LOSS: 1.5115 | ACC 0.4812
TRAIN: EPOCH 0001/0010 | BATCH 0128/0938 | LOSS: 1.5075 | ACC 0.4830
TRAIN: EPOCH 0001/0010 | BATCH 0129/0938 | LOSS: 1.5045 | ACC 0.4847
TRAIN: EPOCH 0001/0010 | BATCH 0130/0938 | LOSS: 1.4990 | ACC 0.4865
TRAIN: EPOCH 0001/0010 | BATCH 0131/0938 | LOSS: 1.4934 | ACC 0.4889
TRAIN: EPOCH 0001/0010 | BATCH 0132/0938 | LOSS: 1.4892 | ACC 0.4908
TRAIN: EPOCH 0001/0010 | BATCH 0133/0938 | LOSS: 1.4835 | ACC 0.4928
TRAIN: EPOCH 0001/0010 | BATCH 013

TRAIN: EPOCH 0001/0010 | BATCH 0239/0938 | LOSS: 1.1449 | ACC 0.6169
TRAIN: EPOCH 0001/0010 | BATCH 0240/0938 | LOSS: 1.1425 | ACC 0.6174
TRAIN: EPOCH 0001/0010 | BATCH 0241/0938 | LOSS: 1.1396 | ACC 0.6185
TRAIN: EPOCH 0001/0010 | BATCH 0242/0938 | LOSS: 1.1371 | ACC 0.6194
TRAIN: EPOCH 0001/0010 | BATCH 0243/0938 | LOSS: 1.1352 | ACC 0.6200
TRAIN: EPOCH 0001/0010 | BATCH 0244/0938 | LOSS: 1.1337 | ACC 0.6206
TRAIN: EPOCH 0001/0010 | BATCH 0245/0938 | LOSS: 1.1311 | ACC 0.6216
TRAIN: EPOCH 0001/0010 | BATCH 0246/0938 | LOSS: 1.1294 | ACC 0.6223
TRAIN: EPOCH 0001/0010 | BATCH 0247/0938 | LOSS: 1.1264 | ACC 0.6234
TRAIN: EPOCH 0001/0010 | BATCH 0248/0938 | LOSS: 1.1236 | ACC 0.6245
TRAIN: EPOCH 0001/0010 | BATCH 0249/0938 | LOSS: 1.1210 | ACC 0.6254
TRAIN: EPOCH 0001/0010 | BATCH 0250/0938 | LOSS: 1.1189 | ACC 0.6261
TRAIN: EPOCH 0001/0010 | BATCH 0251/0938 | LOSS: 1.1191 | ACC 0.6262
TRAIN: EPOCH 0001/0010 | BATCH 0252/0938 | LOSS: 1.1165 | ACC 0.6270
TRAIN: EPOCH 0001/0010 | BATCH 025

TRAIN: EPOCH 0001/0010 | BATCH 0359/0938 | LOSS: 0.9410 | ACC 0.6902
TRAIN: EPOCH 0001/0010 | BATCH 0360/0938 | LOSS: 0.9398 | ACC 0.6904
TRAIN: EPOCH 0001/0010 | BATCH 0361/0938 | LOSS: 0.9392 | ACC 0.6906
TRAIN: EPOCH 0001/0010 | BATCH 0362/0938 | LOSS: 0.9377 | ACC 0.6910
TRAIN: EPOCH 0001/0010 | BATCH 0363/0938 | LOSS: 0.9360 | ACC 0.6917
TRAIN: EPOCH 0001/0010 | BATCH 0364/0938 | LOSS: 0.9345 | ACC 0.6922
TRAIN: EPOCH 0001/0010 | BATCH 0365/0938 | LOSS: 0.9335 | ACC 0.6925
TRAIN: EPOCH 0001/0010 | BATCH 0366/0938 | LOSS: 0.9323 | ACC 0.6928
TRAIN: EPOCH 0001/0010 | BATCH 0367/0938 | LOSS: 0.9313 | ACC 0.6931
TRAIN: EPOCH 0001/0010 | BATCH 0368/0938 | LOSS: 0.9303 | ACC 0.6935
TRAIN: EPOCH 0001/0010 | BATCH 0369/0938 | LOSS: 0.9290 | ACC 0.6939
TRAIN: EPOCH 0001/0010 | BATCH 0370/0938 | LOSS: 0.9275 | ACC 0.6943
TRAIN: EPOCH 0001/0010 | BATCH 0371/0938 | LOSS: 0.9265 | ACC 0.6947
TRAIN: EPOCH 0001/0010 | BATCH 0372/0938 | LOSS: 0.9253 | ACC 0.6952
TRAIN: EPOCH 0001/0010 | BATCH 037

...

TRAIN: EPOCH 0010/0010 | BATCH 0384/0938 | LOSS: 0.1486 | ACC 0.9568
TRAIN: EPOCH 0010/0010 | BATCH 0385/0938 | LOSS: 0.1489 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0386/0938 | LOSS: 0.1490 | ACC 0.9566
TRAIN: EPOCH 0010/0010 | BATCH 0387/0938 | LOSS: 0.1489 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0388/0938 | LOSS: 0.1487 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0389/0938 | LOSS: 0.1493 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0390/0938 | LOSS: 0.1494 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0391/0938 | LOSS: 0.1494 | ACC 0.9566
TRAIN: EPOCH 0010/0010 | BATCH 0392/0938 | LOSS: 0.1491 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0393/0938 | LOSS: 0.1490 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 0394/0938 | LOSS: 0.1493 | ACC 0.9566
TRAIN: EPOCH 0010/0010 | BATCH 0395/0938 | LOSS: 0.1496 | ACC 0.9566
TRAIN: EPOCH 0010/0010 | BATCH 0396/0938 | LOSS: 0.1495 | ACC 0.9566
TRAIN: EPOCH 0010/0010 | BATCH 0397/0938 | LOSS: 0.1493 | ACC 0.9567
TRAIN: EPOCH 0010/0010 | BATCH 039

TRAIN: EPOCH 0010/0010 | BATCH 0508/0938 | LOSS: 0.1467 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0509/0938 | LOSS: 0.1466 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0510/0938 | LOSS: 0.1466 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0511/0938 | LOSS: 0.1466 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0512/0938 | LOSS: 0.1466 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0513/0938 | LOSS: 0.1468 | ACC 0.9572
TRAIN: EPOCH 0010/0010 | BATCH 0514/0938 | LOSS: 0.1469 | ACC 0.9572
TRAIN: EPOCH 0010/0010 | BATCH 0515/0938 | LOSS: 0.1469 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0516/0938 | LOSS: 0.1467 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0517/0938 | LOSS: 0.1467 | ACC 0.9572
TRAIN: EPOCH 0010/0010 | BATCH 0518/0938 | LOSS: 0.1473 | ACC 0.9571
TRAIN: EPOCH 0010/0010 | BATCH 0519/0938 | LOSS: 0.1473 | ACC 0.9571
TRAIN: EPOCH 0010/0010 | BATCH 0520/0938 | LOSS: 0.1473 | ACC 0.9570
TRAIN: EPOCH 0010/0010 | BATCH 0521/0938 | LOSS: 0.1472 | ACC 0.9571
TRAIN: EPOCH 0010/0010 | BATCH 052

TRAIN: EPOCH 0010/0010 | BATCH 0627/0938 | LOSS: 0.1465 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0628/0938 | LOSS: 0.1463 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0629/0938 | LOSS: 0.1462 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0630/0938 | LOSS: 0.1460 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0631/0938 | LOSS: 0.1460 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0632/0938 | LOSS: 0.1460 | ACC 0.9576
TRAIN: EPOCH 0010/0010 | BATCH 0633/0938 | LOSS: 0.1462 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0634/0938 | LOSS: 0.1462 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0635/0938 | LOSS: 0.1463 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0636/0938 | LOSS: 0.1461 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0637/0938 | LOSS: 0.1462 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0638/0938 | LOSS: 0.1463 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0639/0938 | LOSS: 0.1462 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0640/0938 | LOSS: 0.1462 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 064

TRAIN: EPOCH 0010/0010 | BATCH 0749/0938 | LOSS: 0.1465 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0750/0938 | LOSS: 0.1463 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0751/0938 | LOSS: 0.1464 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0752/0938 | LOSS: 0.1464 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0753/0938 | LOSS: 0.1463 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0754/0938 | LOSS: 0.1462 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0755/0938 | LOSS: 0.1462 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0756/0938 | LOSS: 0.1461 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0757/0938 | LOSS: 0.1465 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0758/0938 | LOSS: 0.1465 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0759/0938 | LOSS: 0.1464 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0760/0938 | LOSS: 0.1463 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0761/0938 | LOSS: 0.1462 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 0762/0938 | LOSS: 0.1462 | ACC 0.9573
TRAIN: EPOCH 0010/0010 | BATCH 076

TRAIN: EPOCH 0010/0010 | BATCH 0872/0938 | LOSS: 0.1458 | ACC 0.9576
TRAIN: EPOCH 0010/0010 | BATCH 0873/0938 | LOSS: 0.1458 | ACC 0.9576
TRAIN: EPOCH 0010/0010 | BATCH 0874/0938 | LOSS: 0.1459 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0875/0938 | LOSS: 0.1459 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0876/0938 | LOSS: 0.1459 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0877/0938 | LOSS: 0.1462 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0878/0938 | LOSS: 0.1462 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0879/0938 | LOSS: 0.1465 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0880/0938 | LOSS: 0.1464 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0881/0938 | LOSS: 0.1464 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0882/0938 | LOSS: 0.1466 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0883/0938 | LOSS: 0.1466 | ACC 0.9574
TRAIN: EPOCH 0010/0010 | BATCH 0884/0938 | LOSS: 0.1464 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 0885/0938 | LOSS: 0.1466 | ACC 0.9575
TRAIN: EPOCH 0010/0010 | BATCH 088